# prologue

### set up notebook and load package

In [1]:
# load what we need
import numpy as np
import CHIRPS.datasets as ds
import CHIRPS.reproducible as rp
import multiprocessing as mp

## Sensitivity Analysis

In [ ]:
# Optional Memory and Computation Cost Management
# CHIRPS is time economical but memory intensive to compute for lots of instances at once.
forest_walk_async=True
chirps_explanation_async=False
n_cores = mp.cpu_count()-6

# How many instances from the test set do you want to explain?
# A number bigger than the test set will be interpreted as 'all'
n_instances = 1000
start_instance = 0 # here can opt to start at a specific instance, diagnostic if something crashes

model = 'RandomForest'
# model = 'AdaBoost1'
# model = 'AdaBoost2'
# model = 'GBM'

# prepare data
datasets = [
#         ds.adult,
#         ds.bankmark,
        ds.car,
#         ds.cardio,
#         ds.credit,
#         ds.german,
#         ds.lending_tiny_samp,
#         ds.nursery,
#        ds.rcdv,
       ]

# location to save results
project_dir = '/datadisk/whiteboxing/benchmarks'
# project_dir = 'V:\\whiteboxing\\tests' # defaults to a directory "whiteboxing" in the working directory
# project_dir = 'C:\\Users\\Crutt\\Documents\\whiteboxing\\tests'

random_state_splits = 123 # change this if you want to try different splits of the data into test / train
random_state_rf = 123 # change this if you want to try with different forest construction
random_state_exp = 123 # change this if you want to try with different runs of the explainer algorithm (affects bootstrap eval)

verbose = True

# CHIRPS default parameters - see papers for details
merging_bootstraps = 20 # how many training bootstraps to test improvement in growing rule?
pruning_bootstraps = 20 # how many training bootstraps to test deterioration in pruning rule?
delta = 0.1 # pruning deterioration tolerance paramater

tuning = {'override' : False}

if model == 'RandomForest':
    tuning.update({'grid' : {'n_estimators': [(i + 1) * 200 for i in range(8)], 'max_depth' : [32]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=random_state_rf,
                                           random_state_splits=random_state_splits,
                                           start_instance=start_instance,
                                           verbose=verbose, n_cores=n_cores)

    alpha_paths = np.tile([0.9, 0.5, 0.1], 24)
    disc_path_bins = np.tile(np.repeat([4, 8], 3), 12)
    score_func = np.tile(np.repeat([5, 3, 1], 6), 4)
    support_paths = np.tile(np.repeat([0.1, 0.05], 18), 2)
    weighting = np.repeat(['kldiv', 'nothing'], 36)

    kwargs_grid = {k : {'alpha_paths' : ap, 'disc_path_bins' : dpb, 'disc_path_eqcounts' : False,
                        'score_func' : sf, 'weighting' : w, 'support_paths' : sp,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : 'majority',
                        'delta' : delta} 
        for k, ap, dpb, sf, w, sp in zip(range(72), alpha_paths, disc_path_bins, score_func, weighting, support_paths)}
    
    
    for kwargs in kwargs_grid:
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'save_sensitivity_path' : 'rf_sensitivity'}
        
        rp.do_benchmarking(bi_copy, verbose, **control)
        
elif model in ('AdaBoost1', 'AdaBoost2'):
    if model == 'AdaBoost1':
        algo = 'SAMME'
        save_sensitivity_path = 'ada1_sensitivity'
        support_paths = np.tile(np.tile([0.05, 0.02, 0.01], 16), 2)
        n_kwargs = 36
    else:
        algo = 'SAMME.R'
        save_sensitivity_path = 'ada2_sensitivity'
        support_paths = np.tile(np.tile([0.005, 0.002, 0.001], 16), 2)
        n_kwargs = 72
        
    max_depth = [i for i in range(1, 5)]
    tuning.update({'grid' : {'base_estimator' : [rp.DecisionTreeClassifier(max_depth=d) for d in max_depth],
                            'n_estimators': [(i + 1) * 200 for i in range(8)], 'algorithm': [algo]}})
    benchmark_items = rp.benchmarking_prep(datasets, model, tuning, project_dir,
                                           random_state=123, random_state_splits=123,
                                           start_instance=start_instance,
                                           verbose=verbose, n_cores=n_cores)
    
    disc_path_bins = np.tile(np.tile(np.repeat([4, 8], 3), 6), 2)
    disc_path_eqcounts = np.tile(np.tile(np.repeat([True, False], 6), 6), 2)
    weighting = np.tile(np.repeat(['chisq', 'kldiv', 'nothing'], 12), 2)
    which_trees = np.repeat(['majority', 'conf_weighted'], 36)

    kwargs_grid = {k : {'paths_lengths_threshold' : 5, 'alpha_paths' : 0.0,
                        'disc_path_bins' : dpb, 'disc_path_eqcounts' : dpeq,
                        'score_func' : 1, 'weighting' : w, 'support_paths' : sp,
                        'merging_bootstraps' : merging_bootstraps,
                        'pruning_bootstraps' : pruning_bootstraps,
                        'which_trees' : wchtr,
                        'delta' : delta} 
    for k, dpb, dpeq, w, sp, wchtr \
                   in zip(range(n_kwargs), disc_path_bins, disc_path_eqcounts, weighting, support_paths, which_trees)}

    for kwargs in kwargs_grid: # range(24, 36): #
        bi_copy = rp.deepcopy(benchmark_items) # to avoid running down the internal counters
        control = {'method' : 'CHIRPS', 'model' : model,
                    'n_instances' : n_instances,
                    'random_state' : random_state_exp,
                    'kwargs' : kwargs_grid[kwargs],
                    'forest_walk_async' : forest_walk_async,
                    'chirps_explanation_async' : chirps_explanation_async,
                    'save_sensitivity_path' : save_sensitivity_path}
 
        rp.do_benchmarking(bi_copy, verbose, **control)
else:
    print(model + ' not implemented')

Preprocessing car data and model for car with random state = 123
Split data into main train-test and build forest
Train main model
using previous tuning parameters


/home/julianhatwell/anaconda3/envs/B3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Best OOB Accuracy Estimate during tuning: 0.9843
Best parameters:{'max_depth': 32.0, 'n_estimators': 800, 'oob_score': True, 'random_state': 123}


Beginning benchmark for car data.
Hyper-parameter settings: {'alpha_paths': 0.9, 'disc_path_bins': 4, 'disc_path_eqcounts': False, 'score_func': 5, 'weighting': 'kldiv', 'support_paths': 0.1, 'merging_bootstraps': 20, 'pruning_bootstraps': 20, 'which_trees': 'majority', 'delta': 0.1}

Prepare Unseen Data and Predictions for CHIRPS benchmark
Walking forest for 518 instances... (please wait)
Forest Walk with async = True
Forest Walk time elapsed: 18.7790 seconds

Running CHIRPS on a batch of 518 instances... (please wait)
Working on CHIRPS for instance 0 of 518
as_chirps for batch_idx 0
start mining for batch_idx 0
found 307 patterns from 625 for batch_idx 0
start score sort for batch_idx 0 (307) patterns
start merge rule for batch_idx 0 (307) patterns
[('buying_low', False, 0.5), ('maint_vhigh', True, 0.5), ('persons_4', False, 0.5), ('safet

start merge rule for batch_idx 17 (372) patterns
[('lug_boot_small', True, 0.5), ('maint_med', False, 0.5), ('persons_more', False, 0.5), ('safety_low', True, 0.5)]
0.9375 0.03787820418981285 0.4040328136011205 0.27609227311785284
merge complete for batch_idx 17 (372) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18
found 358 patterns from 540 for batch_idx 18
start score sort for batch_idx 18 (358) patterns
start merge rule for batch_idx 18 (358) patterns
[('lug_boot_small', True, 0.5), ('maint_low', False, 0.5), ('persons_more', False, 0.5), ('safety_low', True, 0.5)]
0.9591836734693877 0.038746908491343775 0.42885110341740806 0.3520782396088016
merge complete for batch_idx 18 (358) patterns
start get explainer for batch_idx 18
as_chirps for batch_idx 19
start mining for batch_idx 19
found 160 patterns from 775 for batch_idx 19
start score sort for batch_idx 19 (160) patterns
start merge rule for batch_idx 19 (160) patterns
[('per

[('persons_2', False, 0.5)]
0.9951100244498777 0.33553173948887055 0.634616331264406 0.5612183353437878
merge complete for batch_idx 37 (23) patterns
start get explainer for batch_idx 37
as_chirps for batch_idx 38
start mining for batch_idx 38
found 210 patterns from 683 for batch_idx 38
start score sort for batch_idx 38 (210) patterns
start merge rule for batch_idx 38 (210) patterns
[('maint_high', True, 0.5), ('maint_vhigh', True, 0.5), ('persons_4', False, 0.5), ('safety_high', False, 0.5)]
0.9705882352941176 0.05441055234954658 0.3481752343812949 0.3349297148685198
merge complete for batch_idx 38 (210) patterns
start get explainer for batch_idx 38
as_chirps for batch_idx 39
start mining for batch_idx 39
found 461 patterns from 523 for batch_idx 39
start score sort for batch_idx 39 (461) patterns
start merge rule for batch_idx 39 (461) patterns
[('buying_high', True, 0.5), ('buying_vhigh', True, 0.5), ('persons_2', True, 0.5), ('safety_low', True, 0.5), ('lug_boot_small', True, 0.5)

...

start merge rule for batch_idx 268 (22) patterns
[('persons_2', False, 0.5)]
0.9951100244498777 0.33553173948887055 0.5661566826604365 0.43012580754845303
merge complete for batch_idx 268 (22) patterns
start get explainer for batch_idx 268
as_chirps for batch_idx 269
start mining for batch_idx 269
found 162 patterns from 717 for batch_idx 269
start score sort for batch_idx 269 (162) patterns
start merge rule for batch_idx 269 (162) patterns
[('maint_med', False, 0.5), ('persons_4', False, 0.5), ('safety_high', False, 0.5)]
0.9473684210526315 0.02967848309975268 0.676651305496716 0.5197537772803584
merge complete for batch_idx 269 (162) patterns
start get explainer for batch_idx 269
Working on CHIRPS for instance 270 of 518
as_chirps for batch_idx 270
start mining for batch_idx 270
found 257 patterns from 737 for batch_idx 270
start score sort for batch_idx 270 (257) patterns
start merge rule for batch_idx 270 (257) patterns
[('lug_boot_small', True, 0.5), ('maint_med', False, 0.5), ('p

found 141 patterns from 743 for batch_idx 289
start score sort for batch_idx 289 (141) patterns
start merge rule for batch_idx 289 (141) patterns
[('buying_vhigh', True, 0.5), ('maint_vhigh', True, 0.5), ('persons_4', False, 0.5), ('safety_high', False, 0.5)]
0.9733333333333334 0.060181368507831824 0.5141007299582512 0.4342792685822845
merge complete for batch_idx 289 (141) patterns
start get explainer for batch_idx 289
Working on CHIRPS for instance 290 of 518
as_chirps for batch_idx 290
start mining for batch_idx 290
found 9 patterns from 800 for batch_idx 290
start score sort for batch_idx 290 (9) patterns
start merge rule for batch_idx 290 (9) patterns
[('safety_low', False, 0.5)]
0.9949367088607595 0.3239901071723001 0.34320973668957927 0.38971499380421315
merge complete for batch_idx 290 (9) patterns
start get explainer for batch_idx 290
as_chirps for batch_idx 291
start mining for batch_idx 291
found 24 patterns from 779 for batch_idx 291
start score sort for batch_idx 291 (24) 

found 349 patterns from 667 for batch_idx 309
start score sort for batch_idx 309 (349) patterns
start merge rule for batch_idx 309 (349) patterns
[('buying_high', False, 0.5), ('lug_boot_small', False, 0.5)]
0.8636363636363636 0.0857657859069124 0.22624211235329844 0.28650943131026296
merge complete for batch_idx 309 (349) patterns
start get explainer for batch_idx 309
Working on CHIRPS for instance 310 of 518
as_chirps for batch_idx 310
start mining for batch_idx 310
found 9 patterns from 799 for batch_idx 310
start score sort for batch_idx 310 (9) patterns
start merge rule for batch_idx 310 (9) patterns
[('safety_low', False, 0.5)]
0.9949367088607595 0.3239901071723001 0.3432096814908 0.3889234598630989
merge complete for batch_idx 310 (9) patterns
start get explainer for batch_idx 310
as_chirps for batch_idx 311
start mining for batch_idx 311
found 19 patterns from 781 for batch_idx 311
start score sort for batch_idx 311 (19) patterns
start merge rule for batch_idx 311 (19) patterns

found 29 patterns from 786 for batch_idx 328
start score sort for batch_idx 328 (29) patterns
start merge rule for batch_idx 328 (29) patterns
[('persons_2', False, 0.5)]
0.9951100244498777 0.33553173948887055 0.35806944360436543 0.33054599114608957
merge complete for batch_idx 328 (29) patterns
start get explainer for batch_idx 328
as_chirps for batch_idx 329
start mining for batch_idx 329
found 224 patterns from 538 for batch_idx 329
start score sort for batch_idx 329 (224) patterns
start merge rule for batch_idx 329 (224) patterns
[('buying_vhigh', False, 0.5), ('maint_high', False, 0.5)]
0.9733333333333334 0.060181368507831824 0.2946888342768842 0.33341835433352024
merge complete for batch_idx 329 (224) patterns
start get explainer for batch_idx 329
Working on CHIRPS for instance 330 of 518
as_chirps for batch_idx 330
start mining for batch_idx 330
found 107 patterns from 786 for batch_idx 330
start score sort for batch_idx 330 (107) patterns
start merge rule for batch_idx 330 (107

[('persons_2', False, 0.5)]
0.9951100244498777 0.33553173948887055 0.6139160422457007 0.4636655948553054
merge complete for batch_idx 348 (9) patterns
start get explainer for batch_idx 348
as_chirps for batch_idx 349
start mining for batch_idx 349
found 250 patterns from 637 for batch_idx 349
start score sort for batch_idx 349 (250) patterns
start merge rule for batch_idx 349 (250) patterns
[('buying_high', True, 0.5), ('buying_vhigh', True, 0.5), ('persons_4', False, 0.5), ('safety_high', False, 0.5)]
0.96875 0.051112943116240726 0.45863198331395694 0.2721893491124261
merge complete for batch_idx 349 (250) patterns
start get explainer for batch_idx 349
Working on CHIRPS for instance 350 of 518
as_chirps for batch_idx 350
start mining for batch_idx 350
found 285 patterns from 641 for batch_idx 350
start score sort for batch_idx 350 (285) patterns
start merge rule for batch_idx 350 (285) patterns
[('lug_boot_small', False, 0.5)]
0.7937649880095923 0.2609442899660458 0.20663193842277844 